In [1]:
!pip install tensorflow==2.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.5.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/RBC-Opponent-Modelling-main/lczero-training-5/lczero-training/tf')
os.listdir()


Mounted at /content/drive


['weights_run2_790389.pb.gz',
 'net.proto',
 'start.sh',
 'update_steps.py',
 'net_to_model.py',
 'chunkparser.py',
 'attention_policy_map.py',
 'make_model.py',
 'lc0_az_policy_map.py',
 'shufflebuffer.py',
 'decode_training.py',
 'chunkparsefunc.py',
 'policy_index.py',
 'model_to_net.py',
 'requirements.txt',
 'train.py',
 'TK-6430 aka 128x10-BPR-64M-6430000.pb.gz',
 'net.py',
 'loading_lc0.ipynb',
 'net.pb.cc',
 'net.pb.h',
 'net_pb2.py',
 'details_from_weights_file.py',
 'LC0_Opponent_Modelling.ipynb',
 'tfprocess.py',
 'leelalogs',
 'configs',
 '__pycache__',
 'model_plot.png']

In [3]:

import yaml
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()

weights_file_name = "TK-6430 aka 128x10-BPR-64M-6430000.pb.gz"
# weights_file_name = "128 blocks x 10 filters weights.pb.gz"


from tfprocess import TFProcess
tfprocess = TFProcess(yaml.safe_load(open("configs/example.yaml").read()))
tfprocess.init_net()
tfprocess.replace_weights(weights_file_name)
from tensorflow import keras



model = tf.keras.Sequential()
model.add(tfprocess.model)
model.add(tf.keras.layers.Softmax())
model.summary()
keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


import tensorflow as tf
# import tensorflow_datasets as tfds
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# device_name = tf.test.gpu_device_name()
# print("Device Name =", device_name)
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))


# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#   try:
#     tf.config.set_visible_devices(gpus[0], 'GPU')
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#   except RuntimeError as e:
#     # Visible devices must be set before GPUs have been initialized
#     print(e)


os.chdir("../..") # nullify effect of previous chdir

import numpy as np
inputs_old = []
outputs_old = []

os.chdir('/content/drive/MyDrive/RBC-Opponent-Modelling-main/Games_Opponents_lastyear')
print(os.listdir())

Blocks: 10
Filters: 128
pblczero.Net.magic: 448
pblczero.Net.license: 
pblczero.EngineVersion.major: 0
pblczero.EngineVersion.minor: 21
pblczero.EngineVersion.patch: 0
pblczero.Format.weights_encoding: LINEAR16
pblczero.NetworkFormat.input: INPUT_CLASSICAL_112_PLANE
pblczero.NetworkFormat.output: OUTPUT_WDL
pblczero.NetworkFormat.network: NETWORK_SE_WITH_HEADFORMAT
pblczero.NetworkFormat.policy: POLICY_CONVOLUTION
pblczero.NetworkFormat.value: VALUE_WDL
pblczero.NetworkFormat.moves_left: MOVES_LEFT_NONE
pblczero.NetworkFormat.default_activation: DEFAULT_ACTIVATION_RELU
pblczero.TrainingParams.training_steps: 6430000
pblczero.TrainingParams.learning_rate: 9.999999747378752e-05
pblczero.TrainingParams.mse_loss: 0.09407615661621094
pblczero.TrainingParams.policy_loss: 1.8348811864852905
pblczero.TrainingParams.accuracy: 59.87109375
pblczero.TrainingParams.lc0_params: 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output

In [4]:
import numpy as np
inputs = []
outputs = []
import os
for f in os.listdir('StrangeFish2_npz')[:900]:
  if not f.endswith('.npz'): continue
  
  loaded = np.load(os.path.join('StrangeFish2_npz',f))
  length = len(loaded['inputs'])
  inputs.extend(loaded['inputs'])
  outputs.extend(loaded['outputs'])
  

inputs = np.array(inputs)
outputs = np.array(outputs)

print(np.shape(inputs),np.shape(outputs))

(18538, 112, 8, 8) (18538, 1858)


In [5]:
perm = np.random.permutation(inputs.shape[0])
inputs = inputs[perm,:,:,:]
outputs = outputs[perm,:]

valx, valy = inputs[int(0.8 * len(inputs)):], outputs[int(0.8 * len(inputs)):]

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

is_value_head = False

In [7]:
stored_outputs = outputs.copy()
value_outputs = None
if is_value_head:
  _value_outputs = model.predict(inputs)
  model.evaluate(x = inputs, y = _value_outputs)
  print(len(_value_outputs))
  print(_value_outputs[0].shape)
  print(len(_value_outputs[0]))
  print(_value_outputs[1].shape)
  print(len(_value_outputs[1]))

  value_outputs = np.zeros([len(inputs), 3])
  value_outputs[:,:] = _value_outputs[1]
  print(value_outputs.shape)
  print(len(outputs))
  print(stored_outputs.shape)
  model.evaluate(x = inputs, y = [stored_outputs, value_outputs])
  print(_value_outputs[0][0])
  print(np.sum(inputs[0]))
  print(stored_outputs[0])
  print(np.sum(stored_outputs[0]))

else:
  model.evaluate(x = valx, y = valy)
  _value_outputs = model.predict(inputs)
  print(_value_outputs[0])
  print(np.sum(_value_outputs[0]))
  print(np.sum(_value_outputs[0] < 0))
  print(np.sum(inputs[0]))
  print(stored_outputs[0])
  print(np.sum(stored_outputs[0]))

del stored_outputs

580/580 [==============================] - 8s 10ms/step
[0. 0. 0. ... 0. 0. 0.]
1.0
0
928.0
[0. 0. 0. ... 0. 0. 0.]
1.0


In [ ]:
history = model.fit(
  inputs[:int(0.8 * len(inputs))],
  outputs[:int(0.8 * len(inputs))],
  validation_data = (inputs[int(0.8 * len(inputs)):], outputs[int(0.8 * len(inputs)):]),
  epochs = 20,
)
  

Epoch 1/20
464/464 [==============================] - 39s 35ms/step - loss: 3.4451 - categorical_accuracy: 0.2444 - val_loss: 8.0432 - val_categorical_accuracy: 0.2729
Epoch 2/20
464/464 [==============================] - 16s 35ms/step - loss: 2.7160 - categorical_accuracy: 0.3195 - val_loss: 3.5531 - val_categorical_accuracy: 0.2902
Epoch 3/20
464/464 [==============================] - 18s 40ms/step - loss: 2.3928 - categorical_accuracy: 0.3759 - val_loss: 2.7753 - val_categorical_accuracy: 0.3385
Epoch 4/20
464/464 [==============================] - 16s 35ms/step - loss: 2.1459 - categorical_accuracy: 0.4363 - val_loss: 4.2515 - val_categorical_accuracy: 0.3398
Epoch 5/20
464/464 [==============================] - 16s 34ms/step - loss: 1.9253 - categorical_accuracy: 0.4829 - val_loss: 2.8565 - val_categorical_accuracy: 0.3433
Epoch 6/20
464/464 [==============================] - 15s 33ms/step - loss: 1.6941 - categorical_accuracy: 0.5466 - val_loss: 4.0355 - val_categorical_accuracy:

In [ ]:
# model.save('newWeights.pb.gz')

from google.colab import files
import shutil, os
folder_name="SF2"

if os.path.exists(folder_name):
  shutil.rmtree(folder_name)
os.mkdir(folder_name)

if os.path.exists(folder_name+".zip"):
  os.remove(folder_name+".zip")
model.save(folder_name)

shutil.make_archive(folder_name,'zip',folder_name)
files.download(folder_name+".zip")

if os.path.exists(folder_name):
  shutil.rmtree(folder_name)
os.mkdir(folder_name)

if os.path.exists(folder_name+".pb.gz"):
  os.remove(folder_name+".pb.gz")
tfprocess.model=model
tfprocess.save_leelaz_weights(folder_name)
files.download(folder_name+".pb.gz")

In [ ]:

import pickle

with open('history.pickle', 'wb') as handle:
    pickle.dump(dict, handle)

In [ ]:
files.download(folder_name+".pb.gz")

In [ ]:
# !pip install numpy==1.24.2

# import numpy as np
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(2, 2)
# axs[0, 0].plot(list(map(lambda x: x+1, range(len(history.history['loss'])))),  history.history['loss'])
# axs[0, 0].set_title('Axis [0, 0]')
# axs[0, 1].plot(list(map(lambda x: x+1, range(len(history.history['categorical_accuracy'])))),  history.history['categorical_accuracy'])
# axs[0, 1].set_title('Axis [0, 1]')
# axs[1, 0].plot(list(map(lambda x: x+1, range(len(history.history['val_loss'])))),  history.history['val_loss'])
# axs[1, 0].set_title('Axis [1, 0]')
# axs[1, 1].plot(list(map(lambda x: x+1, range(len(history.history['val_categorical_accuracy'])))),  history.history['val_categorical_accuracy'])
# axs[1, 1].set_title('Axis [1, 1]')

# # Hide x labels and tick labels for top plots and y ticks for right plots.
# for ax in axs.flat:
#     ax.label_outer()

# # plt.title('Training Loss')
# # plt.xlabel('epochs')
# # plt.ylabel('Cross Entropy loss')
# # plt.plot(list(map(lambda x: x+1, range(len(history.history['loss'])))),  history.history['loss'])
# # plt.show()
# # dict = {}
# # dict['loss'] = history.history['loss']

# # plt.title('Training Accuracy')
# # plt.xlabel('epochs')
# # plt.ylabel('Accuracy')
# # plt.plot(list(map(lambda x: x+1, range(len(history.history['categorical_accuracy'])))),  history.history['categorical_accuracy'])
# # dict['ca'] = history.history['categorical_accuracy']
# # plt.show()

# # plt.title('Validation Loss')
# # plt.xlabel('epochs')
# # plt.ylabel('Cross Entropy loss')
# # plt.plot(list(map(lambda x: x+1, range(len(history.history['val_loss'])))),  history.history['val_loss'])
# # plt.show()
# # dict['vl'] = history.history['val_loss']

# # plt.title('Validation Accuracy')
# # plt.xlabel('epochs')
# # plt.ylabel('Accuracy')
# # plt.plot(list(map(lambda x: x+1, range(len(history.history['val_categorical_accuracy'])))),  history.history['val_categorical_accuracy'])
# # dict['vca'] = history.history['val_categorical_accuracy']
# # plt.show()